In [23]:
import pandas as pd
import numpy as np
import mne
import os
import tkinter as tk
from tkinter import filedialog, messagebox
from tkinter import ttk
from scipy.io import savemat

class EEGDataExtractorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("EEG Data Extractor")
        self.root.geometry("800x600")

        button_frame = tk.Frame(root)
        button_frame.pack(pady=10)

        # 添加加载TSV文件的下拉菜单
        self.load_tsv_mode = tk.StringVar()
        self.load_tsv_mode.set("单个加载")
        tsv_mode_menu = ttk.Combobox(button_frame, textvariable=self.load_tsv_mode, values=["单个加载", "批量加载"])
        tsv_mode_menu.pack(side=tk.LEFT, padx=5)

        self.load_tsv_button = tk.Button(button_frame, text="加载TSV文件", command=self.load_tsv_files)
        self.load_tsv_button.pack(side=tk.LEFT, padx=5)

        # 添加加载EEG文件的下拉菜单
        self.load_eeg_mode = tk.StringVar()
        self.load_eeg_mode.set("单个加载")
        eeg_mode_menu = ttk.Combobox(button_frame, textvariable=self.load_eeg_mode, values=["单个加载", "批量加载"])
        eeg_mode_menu.pack(side=tk.LEFT, padx=5)

        self.load_eeg_button = tk.Button(button_frame, text="加载EEG文件", command=self.load_eeg_files, state=tk.DISABLED)
        self.load_eeg_button.pack(side=tk.LEFT, padx=5)

        self.eeg_format_label = tk.Label(root, text="选择EEG数据格式:")
        self.eeg_format_label.pack(pady=5)

        self.eeg_format = tk.StringVar()
        self.eeg_format.set(None)

        self.eeg_format_combobox = ttk.Combobox(root, textvariable=self.eeg_format, values=["edf", "fif", "vhdr", "bdf"])
        self.eeg_format_combobox.bind("<<ComboboxSelected>>", self.on_eeg_format_selected)
        self.eeg_format_combobox.pack(pady=5)

        # 添加保存格式下拉菜单
        self.save_format_label = tk.Label(root, text="选择保存格式:")
        self.save_format_label.pack(pady=5)

        self.save_format = tk.StringVar()
        self.save_format.set("npy")  # 默认保存格式为NPY

        self.save_format_combobox = ttk.Combobox(root, textvariable=self.save_format, values=["npy", "mat"])
        self.save_format_combobox.pack(pady=5)

        self.process_button = tk.Button(root, text="处理数据", command=self.process_data)
        self.process_button.pack(pady=10)

        self.batch_process_button = tk.Button(root, text="批量处理", command=self.batch_process)
        self.batch_process_button.pack(pady=10)

        self.close_button = tk.Button(root, text="关闭", command=self.close_app)
        self.close_button.pack(pady=10)

        self.save_dirs = {}
        self.tsv_file_paths = []
        self.eeg_file_paths = []
        self.output_dir = None

    def load_tsv_files(self):
        mode = self.load_tsv_mode.get()
        if mode == "单个加载":
            tsv_file_paths = filedialog.askopenfilename(filetypes=[("TSV files", "*.tsv")])
            if tsv_file_paths:
                self.tsv_file_paths = [tsv_file_paths]
                self.load_eeg_button.config(state=tk.NORMAL)
                self.create_trial_type_buttons(tsv_file_paths)
                messagebox.showinfo("信息", f"已加载TSV文件: {tsv_file_paths}")
        elif mode == "批量加载":
            tsv_file_paths = filedialog.askopenfilenames(filetypes=[("TSV files", "*.tsv")])
            if tsv_file_paths:
                self.tsv_file_paths = list(tsv_file_paths)
                self.load_eeg_button.config(state=tk.NORMAL)
                self.show_file_correspondence()
                messagebox.showinfo("信息", f"已批量加载 {len(tsv_file_paths)} 个TSV文件")
                self.create_trial_type_buttons(tsv_file_paths)

    def load_eeg_files(self):
        eeg_format = self.eeg_format.get()
        if not eeg_format:
            messagebox.showerror("错误", "请先选择EEG数据格式")
            return

        mode = self.load_eeg_mode.get()
        filetypes = {
            'edf': ("EDF files", "*.edf"),
            'fif': ("FIF files", "*.fif"),
            'vhdr': ("BrainVision files", "*.vhdr"),
            'bdf': ("BDF files", "*.bdf")
        }

        if mode == "单个加载":
            eeg_file_path = filedialog.askopenfilename(filetypes=[filetypes.get(eeg_format, ("All files", "*.*"))])
            if eeg_file_path:
                self.eeg_file_paths = [eeg_file_path]
                messagebox.showinfo("信息", f"已加载EEG文件: {eeg_file_path}")
        elif mode == "批量加载":
            eeg_file_paths = filedialog.askopenfilenames(filetypes=[filetypes.get(eeg_format, ("All files", "*.*"))])
            if eeg_file_paths:
                self.eeg_file_paths = list(eeg_file_paths)
                self.show_file_correspondence()
                messagebox.showinfo("信息", f"已批量加载 {len(eeg_file_paths)} 个EEG文件")

    def on_eeg_format_selected(self, event):
        self.load_eeg_button.config(state=tk.NORMAL)

    def create_trial_type_buttons(self, tsv_file_paths):
        trial_types = set()
        for tsv_file_path in tsv_file_paths:
            df = pd.read_csv(tsv_file_path, sep='\t')
            trial_types.update(df['trial_type'].unique())

        self.save_dirs = {}

        for trial_type in trial_types:
            button = tk.Button(self.root, text=f"选择 {trial_type} 保存目录", command=lambda t=trial_type: self.select_save_dir(t))
            button.pack(pady=5)

    def select_save_dir(self, trial_type=None):
        if trial_type:
            dir_path = filedialog.askdirectory()
            if dir_path:
                self.save_dirs[trial_type] = dir_path
                messagebox.showinfo("信息", f"{trial_type} 保存目录已选择: {dir_path}")

    def show_file_correspondence(self):
        correspondence_window = tk.Toplevel(self.root)
        correspondence_window.title("文件对应关系")
        correspondence_window.geometry("600x400")

        text_area = tk.Text(correspondence_window, wrap=tk.NONE)
        text_area.pack(expand=True, fill=tk.BOTH)

        text_area.insert(tk.END, "TSV文件 - EEG文件\n")
        text_area.insert(tk.END, "-" * 40 + "\n")

        for tsv_file_path, eeg_file_path in zip(self.tsv_file_paths, self.eeg_file_paths):
            tsv_filename = os.path.basename(tsv_file_path)
            eeg_filename = os.path.basename(eeg_file_path)
            text_area.insert(tk.END, f"{tsv_filename} - {eeg_filename}\n")

    def process_data(self):
        if not self.tsv_file_paths or not self.eeg_file_paths:
            messagebox.showerror("错误", "请确保已加载TSV文件和EEG文件")
            return

        if len(self.tsv_file_paths) != len(self.eeg_file_paths):
            messagebox.showerror("错误", "加载的TSV文件和EEG文件数量不匹配")
            return

        if not self.save_dirs:
            messagebox.showerror("错误", "请先选择所有试验类型的保存目录")
            return

        eeg_format = self.eeg_format.get()
        save_format = self.save_format.get()

        try:
            for tsv_idx, (tsv_file_path, eeg_file_path) in enumerate(zip(self.tsv_file_paths, self.eeg_file_paths)):
                events_df = pd.read_csv(tsv_file_path, sep='\t')
                onsets = events_df['onset'].values
                durations = events_df['duration'].values
                trial_types = events_df['trial_type'].values

                if eeg_format == 'edf':
                    eeg_data = mne.io.read_raw_edf(eeg_file_path, preload=True)
                elif eeg_format == 'fif':
                    eeg_data = mne.io.read_raw_fif(eeg_file_path, preload=True)
                elif eeg_format == 'vhdr':
                    eeg_data = mne.io.read_raw_brainvision(eeg_file_path, preload=True)
                elif eeg_format == 'bdf':
                    eeg_data = mne.io.read_raw_bdf(eeg_file_path, preload=True)
                else:
                    raise ValueError("不支持的EEG文件格式")

                sfreq = eeg_data.info['sfreq']

                for i, (onset, duration, trial_type) in enumerate(zip(onsets, durations, trial_types)):
                    start_idx = int(onset * sfreq)
                    end_idx = int((onset + duration) * sfreq)
                    trial_data = eeg_data.get_data(start=start_idx, stop=end_idx)

                    if trial_type in self.save_dirs:
                        # 根据TSV文件的顺序和试验类型命名文件
                        tsv_basename = os.path.basename(tsv_file_path).split('.')[0]
                        if save_format == 'npy':
                            file_name = f'{tsv_idx + 1}_{tsv_basename}_{trial_type}_trial_{i}.npy'
                            file_path = os.path.join(self.save_dirs[trial_type], file_name)
                            np.save(file_path, trial_data)
                        elif save_format == 'mat':
                            file_name = f'{tsv_idx + 1}_{tsv_basename}_{trial_type}_trial_{i}.mat'
                            file_path = os.path.join(self.save_dirs[trial_type], file_name)
                            savemat(file_path, {'data': trial_data})
                        print(f'Saved trial {i} ({trial_type}) to {file_path}')
                    else:
                        messagebox.showwarning("警告", f"未为 trial_type '{trial_type}' 选择保存目录")

            messagebox.showinfo("完成", "所有试验数据已成功保存。")
        except Exception as e:
            messagebox.showerror("错误", f"处理数据时出现错误: {str(e)}")

    def batch_process(self):
        self.process_data()

    def close_app(self):
        self.root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = EEGDataExtractorApp(root)
    root.mainloop()


Extracting EDF parameters from C:\Users\pc\Desktop\李佳\sub-001_ses-03_task_motorimagery_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 98999  =      0.000 ...   395.996 secs...
Saved trial 0 (right) to C:/Users/pc/Desktop/李佳/right\1_sub-001_ses-03_task_motorimagery_events_right_trial_0.npy
Saved trial 1 (left) to C:/Users/pc/Desktop/李佳/left\1_sub-001_ses-03_task_motorimagery_events_left_trial_1.npy
Saved trial 2 (right) to C:/Users/pc/Desktop/李佳/right\1_sub-001_ses-03_task_motorimagery_events_right_trial_2.npy
Saved trial 3 (right) to C:/Users/pc/Desktop/李佳/right\1_sub-001_ses-03_task_motorimagery_events_right_trial_3.npy
Saved trial 4 (right) to C:/Users/pc/Desktop/李佳/right\1_sub-001_ses-03_task_motorimagery_events_right_trial_4.npy
Saved trial 5 (right) to C:/Users/pc/Desktop/李佳/right\1_sub-001_ses-03_task_motorimagery_events_right_trial_5.npy
Saved trial 6 (left) to C:/Users/pc/Desktop/李佳/left\1_sub-001_ses-03_task_motorima